<a href="https://colab.research.google.com/github/robotics-upo/rva-course-material/blob/master/imageprocessingbasics/templatematching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template Matching in OpenCV

In this lab session we will use the following tools:

*   **OpenCV**: http://opencv.org
*   **Numpy**, for handling multidimensional arrays (like images): https://numpy.org/
*   **Matplotlib**, library for visualization in python: https://matplotlib.org/

We will use the 3.x version of OpenCV’s API. We will intensively refer to the documentation (https://opencv-python-tutroals.readthedocs.io/en/latest/index.html)

In [ ]:
#OpenCV module
import cv2

#Numpy module
import numpy as np

#We can use OpenCV in Colab, but not its functions for creating plots
#We use matplotlib for generating plots
from matplotlib import pyplot as plt

#We use the library scikit to read images from url 
#In OpenCV, the function to read from file is cv2.imread
from skimage import io

One function that we have seen in theory is **template matching**. 

The objective is to look for a template on an image, and
determine the most likely location of the template.

For instance, we may want to look, in the following **image**

In [ ]:
#Image
im = io.imread('https://robotics.upo.es/~lmercab/rva/waldo4.jpg')

plt.figure(figsize=(10,10))
plt.imshow(im)

the next **template**:

In [ ]:
#Template
temp = io.imread('https://robotics.upo.es/~lmercab/rva/temp.jpg')

plt.imshow(temp)

#Dimensions of images
print('Image', im.shape)
print('Template', temp.shape)

OpenCV has a function to look for a pattern on an image:

```
result = cv2.matchTemplate(image, templ, method [, result [, mask]])
```

*   http://docs.opencv.org/modules/imgproc/doc/object_detection.html?highlight=matchtemplate#matchtemplate


This function returns in result a **similarity map**, in the form of an image. The maximum of this map will indicate where is the pattern located. 



In [ ]:
#Compute the similarity map
sim_map = cv2.matchTemplate(im,temp,cv2.TM_CCORR_NORMED)

#The returned field is a numpy array of real values
print(sim_map.dtype)

plt.figure(figsize=(10,10))
plt.imshow(sim_map)
plt.colorbar(fraction=0.03)

#Check the max and min values
print('Maximum', np.max(sim_map))
print('Minimum', np.min(sim_map))


To look for the maximum (or minimum) of the previous map, we can use the function:

```
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(src [, mask])
```



In [ ]:
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(sim_map)
print('Max. pixel:', max_loc)
print('Min. pixel:', min_loc)

OpenCV has several functions to draw on the images. They are quite useful for debugging and to show results.

* https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_drawing_functions/py_drawing_functions.html#drawing-functions

For instance:

```
cv2.circle(img, center, radius, color, thickness)
```

which draws a circle on the image img, at the position center, with radius radius.

In the same way, it is possible to draw further figures, as a rectangle:

```
cv2.rectangle(img, upperleftpoint, lowerrightpoint, color, thickness)
```

We use them next to show  the result of the former steps 

In [ ]:
#Store the height and width of the template:
h=temp.shape[0]
w=temp.shape[1]

#Make a copy of the original image to draw over it
#This is important. Assignments (A = B) in numpy arrays does not create 
#a new array, but a new reference to the same allocated data
im_draw = im.copy()

#We draw a rectangle of the same size as the template at the maximum location
#bottom_right is the location of the bottom right pixel with respect to that
#maximum
bottom_right = (max_loc[0] + w, max_loc[1] + h)
cv2.rectangle(im_draw,max_loc, bottom_right, 255, 2)

#We draw a circle at the maximum location
cv2.circle(im_draw,max_loc,10,255,2);

plt.figure(figsize=(10,10))
plt.imshow(im_draw)

# Homework #1

Read about and change and play with the similarity metric considered in the function `matchTemplate` and see the results:

*  CV_TM_SQDIFF (in this case the minimum indicates the maximum similarity)
*  CV_TM_SQDIFF_NORMED (normalized version)
*  CV_TM_CCORR
*  CV_TM_CCORR_NORMED
*  CV_TM_CCOEFF
*  CV_TM_CCOEFF_NORMED